In [1]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer
txt = pd.read_excel('数据比较好的.xlsx')
Y = txt['标签']
jieba.load_userdict(Y)
stopwords = [line.strip() for line in open('../input/stopword.txt', 'r', encoding='utf-8').readlines()] 
list_des = txt['商品描述'].tolist()
X = []
for i in range(len(list_des)):
    des = list_des[i].replace(' ','')
    seg_list = jieba.lcut(des)
    word_list = [] 
    for seg in seg_list:
        if seg not in stopwords:
            word_list.append(seg)
    X.append((' ').join(word_list))
txt['商品描述'] = pd.Series(X)  
for i in range(len(txt)):
    length = len(txt.loc[i]['商品描述'].split())
    if length >15:
        txt.drop(index = i,inplace=True)
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(Y)
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
        continue
    elif dic[i]>=30:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)
frame = frame.reset_index(drop=True)
c_num = len(Counter(frame['标签']).keys())
print('数据量：%d'%frame.shape[0],'分类数目：%d'%len(Counter(frame['标签']).keys()))

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hebo\AppData\Local\Temp\jieba.cache
Loading model cost 0.744 seconds.
Prefix dict has been built succesfully.


数据量：40387 分类数目：511


In [2]:
token_dictionary = {}
token_index = 0
for line in frame['商品描述']:
    tokens = line.split()
    for token in tokens:
        if token not in token_dictionary:
            token_dictionary[token] = token_index
            token_index += 1
token_dictionary['</s>'] = token_index

In [3]:
frame_index = pd.DataFrame(columns=['商品描述','标签'])
padding_index = len(token_dictionary) - 1
index = []
for line in frame['商品描述']:
    tokens = line.split()
    line_index = []
    for token in tokens:
        line_index += [token_dictionary[token]]
    for _ in range(len(line_index),15):
        line_index.append(padding_index)
    index.append(line_index)
    
frame_index['商品描述'] = pd.Series(index)
frame_index['标签'] = frame['标签']

In [4]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(frame_index['标签'])

In [5]:
import tensorflow as tf
from tensorflow.contrib import rnn
import random

input_size = 100
timestep_size = 15 
hidden_size = 256
layer_num = 2
class_num = c_num
learning_rate = 0.001
vocabulary_size = len(token_dictionary)
embed_dim = 100
sequence_length = 15

with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')
    #_X = tf.placeholder(tf.float32,[None,1500])
    y = tf.placeholder(tf.int64,[None])
    #x = tf.reshape(_X,[-1,timestep_size,input_size])
    input_data = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_data')               

with tf.name_scope("fw"):
    stacked_rnn_fw = []
    for _ in range(layer_num):
        fw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_fw.append(fw_cell)
    lstm_fw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_fw), output_keep_prob= keep_prob)
    #lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)

with tf.name_scope("bw"):
    stacked_rnn_bw = []
    for _ in range(layer_num):
        bw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_bw.append(bw_cell)
    lstm_bw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_bw), output_keep_prob= keep_prob)
    
with tf.name_scope("embedding_layer"):
    embedding = tf.get_variable("embedding",[vocabulary_size,embed_dim],dtype=tf.float32)
    inputs=tf.nn.embedding_lookup(embedding,input_data)    
#input_data (batch_size , sequence_length)
#inputs shape : (batch_size , sequence_length , embed_dim)

inputs = tf.transpose(inputs, [1,0,2])
#inputs shape : (sequence_length, batch_size, embed_dim)

inputs = tf.reshape(inputs, [-1, embed_dim])

# 转换成list,里面的每个元素是(batch_size, embed_dim)
inputs = tf.split(inputs, sequence_length, 0)     
                      
with tf.name_scope("output"):
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, inputs, dtype=tf.float32)
with tf.name_scope("result"):
    w = tf.Variable(tf.random_uniform([2 * hidden_size, class_num], -1.0, 1.0), name='W')
    b = tf.Variable(tf.constant(0.1,shape=[class_num]), dtype=tf.float32)
    logits = tf.nn.xw_plus_b(outputs[-1], w, b)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits+1e-10,labels = y)
    original_cost_function = tf.reduce_mean(loss)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(original_cost_function)
with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction,y)
    correct_num=tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")

In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(frame_index['商品描述'].tolist(),y_label,test_size = 0.1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(5001):
        _batch_size = 1000
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x =[]
        temp_y = []
        for _ in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%200 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={input_data:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy))
        sess.run(train_op,feed_dict={input_data:temp_x,y:temp_y,keep_prob: 0.5, batch_size: _batch_size})

Iter 1: step 1, training accuracy 0.0034662
Iter 6: step 201, training accuracy 0.755385
Iter 12: step 401, training accuracy 0.778906
Iter 17: step 601, training accuracy 0.77643
Iter 23: step 801, training accuracy 0.77742
Iter 28: step 1001, training accuracy 0.77742
Iter 34: step 1201, training accuracy 0.777915
Iter 39: step 1401, training accuracy 0.77742
Iter 45: step 1601, training accuracy 0.773706
Iter 51: step 1801, training accuracy 0.771231
Iter 56: step 2001, training accuracy 0.782867
Iter 62: step 2201, training accuracy 0.776182
Iter 67: step 2401, training accuracy 0.775935
Iter 73: step 2601, training accuracy 0.775439
Iter 78: step 2801, training accuracy 0.780391
Iter 84: step 3001, training accuracy 0.773954
Iter 89: step 3201, training accuracy 0.775439
Iter 95: step 3401, training accuracy 0.785343
Iter 101: step 3601, training accuracy 0.777668
Iter 106: step 3801, training accuracy 0.776925
Iter 112: step 4001, training accuracy 0.789057
Iter 117: step 4201, t

In [9]:
X_train,X_test,Y_train,Y_test = train_test_split(frame_index['商品描述'].tolist(),y_label,test_size = 0.1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(5001):
        _batch_size = 1000
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x =[]
        temp_y = []
        for _ in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%200 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={input_data:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy))
        sess.run(train_op,feed_dict={input_data:temp_x,y:temp_y,keep_prob: 0.3, batch_size: _batch_size})

Iter 1: step 1, training accuracy 0.00123793
Iter 6: step 201, training accuracy 0.737559
Iter 12: step 401, training accuracy 0.782619
Iter 17: step 601, training accuracy 0.793761
Iter 23: step 801, training accuracy 0.798713
Iter 28: step 1001, training accuracy 0.793761
Iter 34: step 1201, training accuracy 0.799703
Iter 39: step 1401, training accuracy 0.796732


KeyboardInterrupt: 

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
a = CountVectorizer().fit_transform(['你好 啊 不好','怎么样 不怎么'])

In [12]:
print(a)

  (0, 0)	1
  (0, 2)	1
  (1, 1)	1
  (1, 3)	1
